# 数据预处理

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
后面的章节将介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。

In [2]:
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。


In [3]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


/tmp/ipykernel_418/38268100.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  inputs = inputs.fillna(inputs.mean())


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 **[数据操作那节](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/01.preliminaries/ndarray.ipynb)** 中引入的那些张量函数来进一步操作。

In [5]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。

### 1.练习

In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'student.csv')
with open(data_file, 'w') as f:
    f.write('stuID,name,sex,class,grade\n')
    f.write('1,liming,1,5,3\n')
    f.write('2,wangjie,1,NA,2\n')
    f.write('3,liwei,1,1,NA\n')
    f.write('4,zhangliang,1,3,1\n')
    f.write('5,liuming,0,4,NA\n')
    f.write('6,wangqiang,NA,2,4\n')
    f.write('7,zhangwei,1,4,NA\n')
    f.write('8,liqiang,1,5,3\n')

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   stuID        name  sex  class  grade
0      1      liming  1.0    5.0    3.0
1      2     wangjie  1.0    NaN    2.0
2      3       liwei  1.0    1.0    NaN
3      4  zhangliang  1.0    3.0    1.0
4      5     liuming  0.0    4.0    NaN
5      6   wangqiang  NaN    2.0    4.0
6      7    zhangwei  1.0    4.0    NaN
7      8     liqiang  1.0    5.0    3.0


In [3]:
data.drop(data.isnull().sum().idxmax(), axis=1, inplace=True)
data

,stuID,name,sex,class
0,1,liming,1.0,5.0
1,2,wangjie,1.0,NaN
2,3,liwei,1.0,1.0
3,4,zhangliang,1.0,3.0
4,5,liuming,0.0,4.0
5,6,wangqiang,NaN,2.0
6,7,zhangwei,1.0,4.0
7,8,liqiang,1.0,5.0


In [4]:
data.drop('name', axis=1, inplace=True)
data

,stuID,sex,class
0,1,1.0,5.0
1,2,1.0,NaN
2,3,1.0,1.0
3,4,1.0,3.0
4,5,0.0,4.0
5,6,NaN,2.0
6,7,1.0,4.0
7,8,1.0,5.0


In [5]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(round(inputs.mean()))
outputs = outputs.fillna(round(outputs.mean()))
print(inputs)
print(outputs)

   stuID  sex
0      1  1.0
1      2  1.0
2      3  1.0
3      4  1.0
4      5  0.0
5      6  1.0
6      7  1.0
7      8  1.0
0    5.0
1    3.0
2    1.0
3    3.0
4    4.0
5    2.0
6    4.0
7    5.0
Name: class, dtype: float64


In [6]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[1., 1.],
         [2., 1.],
         [3., 1.],
         [4., 1.],
         [5., 0.],
         [6., 1.],
         [7., 1.],
         [8., 1.]], dtype=torch.float64),
 tensor([5., 3., 1., 3., 4., 2., 4., 5.], dtype=torch.float64))

### 习题总结

对student.csv做了删除包含None最多的列，考虑到姓名是一个可以忽略的因素故删除name列，剩下的缺省值填上均值（四舍五入后的均值），最后得到X，y张量，求f，可以理解为求性别和班级存在什么函数关系（班级数值越小，班级水平越高）。